<center><h1>Using Langchain Agents</h1></center>
<hr><hr><hr>

## Langchain Agent:
We've so far create examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os

azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_KEY")
api_version = "2023-05-15"

# For working with AzureOpenAI, in place of model, the deployment name is used
deployment_name = os.getenv("DEPLOYMENT_NAME")

In [3]:
os.environ["OPENAI_API_TYPE"]     = "azure"
os.environ["OPENAI_API_VERSION"]  = api_version
os.environ["OPENAI_API_KEY"]      = api_key

In [4]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_deployment=deployment_name,
)

question = "What is Integral calculus?"

print( llm.invoke( question ) )

content='Integral calculus is a branch of calculus that deals with the concept of integration. It focuses on finding the integral of a function, which represents the area under the curve of the function. The integral of a function can also give information about accumulation, such as the total distance traveled or the total amount of a quantity accumulated over a given interval. Integration is the reverse process of differentiation, and it provides a way to find antiderivatives of functions. The techniques and methods of integral calculus are used in various fields, including physics, engineering, economics, and statistics.'


One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:
1. The retriever we just created. This will let it easily answer questions about a context.
2. A search tool. This will let it easily answer questions that require up to date information.

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [6]:
# Creating the embedding-model instance
from langchain_openai import AzureOpenAIEmbeddings

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version=api_version,
)

In [7]:
# Building the vector index and vectorstore
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [8]:
vector = FAISS.from_documents(documents, embeddings_model)

In [9]:
# This "retriever" will take user "input" and return the related data according to the question, which will be passed as "context" to the "document_chain"
retriever = vector.as_retriever()

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

# Obtained retriever in above case will be used here
# retriever = vector.as_retriever()

history_aware_retriever = create_history_aware_retriever(llm, retriever, prompt)

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


conversation_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, conversation_prompt)

history_aware_retrieval_chain = create_retrieval_chain(history_aware_retriever, document_chain)

- The `history_aware_retrieval_chain` is the chain that is created. In addition to `context` it takes the conversation history into account, for answering questions of user.

## Setting up a tool for the retriever we just created:

In [12]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    history_aware_retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [13]:
# Api key of debanjanbusy account:
os.environ["TAVILY_API_KEY"] = "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"

In [14]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

We can now create a list of the tools we want to work with:

In [15]:
tools = [retriever_tool, search]

- Now that we have the tools, we can create an agent to use them.

`pip install langchainhub`

We can use it to create a predefined prompt.

In [16]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_deployment=deployment_name,
    temperature=0
)

In [17]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

We can now invoke the agent and see how it responds! We can ask it questions about LangSmith:

In [18]:
response1 = agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...


NotFoundError: Error code: 404 - {'error': {'message': 'Unrecognized request argument supplied: functions', 'type': 'invalid_request_error', 'param': None, 'code': None}}